In [1]:
%qtconsole

In [5]:
import numpy as np
import tensorflow as tf
import spektral
import pandas as pd

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GeneralConv
from spektral.layers import GlobalSumPool
from spektral.data import DisjointLoader, Dataset
from spektral.datasets import TUDataset
from spektral.datasets.graphsage import Reddit
from spektral.layers import GraphSageConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from spektral.data import BatchLoader
from tensorflow.keras.layers import Layer, Dense, Activation, Input, Lambda, Concatenate, Dropout, ReLU, Reshape,Flatten

from tensorflow.keras.optimizers import SGD, Adam

In [2]:
#Model creation
#Idea from joewilaj/nbaGNNs

class To_Sparse(Layer):

    def __init__(self,):
        super(To_Sparse,self).__init__()

    def call(self,inputs):

        sparse_t = tf.sparse.from_dense(inputs[0], name = 'adj_mat_sp')

        return sparse_t


class node_pair_xt(Layer):

    def __init__(self):
        super(node_pair_xt,self).__init__()

    def call(self,inputs):
        
        extracted = tf.gather(inputs[1],inputs[0], axis=1)

        return extracted


def LinkPredictor(nodes_n,features_n,ch_gnn1,ch_gnn2,drop_rate):

    #Inputs
    raw_adj = Input(shape=(nodes_n,nodes_n),name='adj_mtx')
    adjmtx = To_Sparse()(raw_adj)
    nodefeat= Input(shape=(nodes_n, features_n), name='feature_mtx')
    #index_train = Input(shape=(2,),dtype = tf.int64, name='index_pair_nodes')
    index_train = [[0,1]]
    #Layers
    conv1 = spektral.layers.GeneralConv(channels= ch_gnn1, name='gcn1')([nodefeat,adjmtx])
    conv2 = spektral.layers.GeneralConv(channels= ch_gnn2, name='gcn2')([conv1,adjmtx])
    extracted = node_pair_xt()([index_train,conv2])
    rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
    
    dense = Dense(64)(rshp)
    dense = Dropout(drop_rate)(dense)
    dense = Dense(12)(dense)
    dense = Dropout(drop_rate)(dense)
    output_layer = Dense(1, 'linear')(dense)
    
    #Model
    model = Model(inputs= [raw_adj, nodefeat ], outputs = output_layer)
       
    return model
   

In [88]:
link_pred = LinkPredictor(nodes_n=23, features_n=12, ch_gnn1=64, ch_gnn2=128, drop_rate=0.3)
link_pred.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 adj_mtx (InputLayer)        [(None, 23, 23)]             0         []                            
                                                                                                  
 feature_mtx (InputLayer)    [(None, 23, 12)]             0         []                            
                                                                                                  
 to__sparse_1 (To_Sparse)    (23, 23)                     0         ['adj_mtx[0][0]']             
                                                                                                  
 gcn1 (GeneralConv)          (None, 23, 64)               2560      ['feature_mtx[0][0]',         
                                                                     'to__sparse_1[0][0]']  

In [ ]:
adj = pd.read_csv('AdjMtx_Covid_Undirected.csv')
adj.columns.values[0] = 'Index'


In [ ]:
df = pd.melt(adj, id_vars=['Index'])

#Split training test 90-10


for rindex in range(df.shape[0]):
    
    temp_row = df.iloc[[rindex]]
    temp_df = df.drop([rindex,])
    temp_df = temp_df.sample(frac = 1)
    temp_df= pd.concat([temp_row, temp_df])
    
    
    temp_df = temp_df.pivot(index='Index', columns='variable', values='value')    
    temp_df = temp_df.to_numpy()
    print(temp_df[0,1])      
    if rindex == 0:
        y_out = np.array(temp_df[0,1])
        temp_df[0,1] = -1
        temp_df = np.expand_dims(temp_df, axis=0)
        x_out=temp_df
        
    else:
        y_out = np.append(y_out, temp_df[0,1])
        temp_df[0,1] = -1
        temp_df = np.expand_dims(temp_df, axis=0)
        x_out=np.concatenate((x_out, temp_df), axis=0)
    
 
#print(y_out)

In [128]:
    temp_row = df.iloc[[0]]
    temp_df = df.drop([0,])
    temp_df = temp_df.sample(frac = 1)
    temp_df= pd.concat([temp_row, temp_df])
    print(temp_df)
    
    temp_df = temp_df.pivot(index='Index', columns='variable', values='value')    
    #temp_df = temp_df.to_numpy()
    print(temp_df)     

       Index variable     value
0    A22893C  A22893C  1.000000
86   T22942G  A23579G  0.045817
435  T23031C  T23018C  0.823529
521  T22930A  T23075C  1.000000
523  T22942G  T23075C  0.996813
..       ...      ...       ...
376  G22895C  T22942A  0.003363
8    G22895C  A22893C  0.000000
267  T22928C  G23401T  0.000000
223  T22942A  G22898A  0.003515
488  C22995G  T23031C  0.982857

[529 rows x 3 columns]
variable   A22893C   A22910G   A23535G   A23579G   C22995A   C22995G  \
Index                                                                  
A22893C   1.000000  0.000000  0.000000  0.000000  0.999506  0.000000   
A22910G   0.000000  1.000000  0.000000  0.000000  1.000000  0.000000   
A23535G   0.000000  0.000000  1.000000  0.000000  1.000000  0.000000   
A23579G   0.000000  0.000000  0.000000  1.000000  1.000000  0.000000   
C22995A   0.256111  0.012413  0.019379  0.014566  1.000000  0.000000   
C22995G   0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   
C23123T   0.0000

In [107]:
print(x_out[10])

[[ 1.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00
   9.99505685e-01  0.00000000e+00  0.00000000e+00  4.78497281e-01
   0.00000000e+00  1.52249135e-01  1.48294612e-03  2.22441918e-02
   9.88630746e-04  9.99505685e-01  1.48294612e-03  0.00000000e+00
   8.20069204e-01  1.54720712e-01  0.00000000e+00  8.43302027e-01
   1.51754820e-01  4.94315373e-04  9.92090954e-01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  7.95918367e-01
   6.12244898e-02  1.83673469e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  6.53061224e-01  0.00000000e+00  0.00000000e+00
   6.12244898e-02  3.36734694e-01  6.12244898e-02  5.91836735e-01
   7.14285714e-02  1.12244898e-01  9.89795918e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  8.23529412e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  9.93464052e-01  0.000

In [95]:
print(x_out.shape)
print(y_out.shape)


link_pred = LinkPredictor(nodes_n=23, features_n=1, ch_gnn1=64, ch_gnn2=128, drop_rate=0.3)
link_pred.compile(loss='mean_squared_error', optimizer= Adam(learning_rate=0.001), metrics=['mean_squared_error'])

Feat_input = np.zeros((529,23,1))

link_pred.fit([x_out, Feat_input],y_out, epochs = 20,batch_size = 10)

#link_pred.predict([Ad_input,Feat_input,index_train])

(529, 23, 23)
(529,)
Epoch 1/20
53/53 [==============================] - 2s 6ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 2/20
53/53 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 3/20
53/53 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 4/20
53/53 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 5/20
53/53 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 6/20
53/53 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 7/20
53/53 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 8/20
53/53 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00
Epoch 9/20
53/53 [==============================] -

In [98]:
link_pred.predict([x_out,Feat_input])

y_out

17/17 [==============================] - 0s 5ms/step


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [91]:
#End2end test on random matrices

Ad_input = np.random.rand(10,23,23)
Ad_input = Ad_input.round()
print(Ad_input.shape)
Feat_input = np.random.rand(10,23,12)

index_train = np.random.randint(1,20, size=(10, 2))

y=np.random.rand(10)
y=y.round()

print(y.shape)

y =tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32')

link_pred = LinkPredictor(nodes_n=23, features_n=12, ch_gnn1=64, ch_gnn2=128, drop_rate=0.3)
link_pred.summary()


link_pred.compile(loss='mean_squared_error', optimizer= opt, metrics=['mean_squared_error'])
link_pred.fit([Ad_input,Feat_input],y, epochs = 10,batch_size = 1)

#link_pred.predict([Ad_input,Feat_input,index_train])

(10, 23, 23)
(10,)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 adj_mtx (InputLayer)        [(None, 23, 23)]             0         []                            
                                                                                                  
 feature_mtx (InputLayer)    [(None, 23, 12)]             0         []                            
                                                                                                  
 to__sparse_2 (To_Sparse)    (23, 23)                     0         ['adj_mtx[0][0]']             
                                                                                                  
 gcn1 (GeneralConv)          (None, 23, 64)               2560      ['feature_mtx[0][0]',         
                                                                     'to_

In [161]:
#Test
Ad_input = np.random.randint(1,100, size=(3,4, 4))
#Ad_input = np.random.rand(3,4,4)
#Ad_input = Ad_input.round()
#print(Ad_input)

index_train = np.random.randint(1,4, size=(3, 2))

print(index_train)

extracted = node([index_train,Ad_input])
#extracted2 = np.take(extracted, [[0,1,2,3,4,5,6,7],[10,11,12,13,14,15,16,17]])
extracted.shape
del(tempindex)
del(out)
print(extracted)

for i in range(index_train.shape[0]):
    #print(i)
    tempindex = np.array(list(range(Ad_input.shape[2]*2)))+(i*Ad_input.shape[2]*2)
    #print(tempindex)
    #print(i)
    if i == 0:
        out = tempindex
    else:
        #print(out)
        #print(tempindex)
        #print(i)
        tempindex = tempindex+ ((Ad_input.shape[2]*2*Ad_input.shape[0])*i)
        out = np.vstack((out, tempindex))
      
        
    
    #print(tempindex)
    
    
    
#print(extracted)
#rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
#print(extracted2)
#print(rshp)
#print(out)
res = np.take(extracted, out)

print(res)
#
#res.shape
#print(out.shape)

a = tf.convert_to_tensor(extracted)
b = tf.convert_to_tensor(out)
# c = a[b] # does not work
#result_tf = tf.gather(a, b)
res2=tf.gather(extracted, out)

[[2 1]
 [1 1]
 [3 1]]
tf.Tensor(
[[[[72 56 67 16]
   [90  5 67 42]]

  [[90  5 67 42]
   [90  5 67 42]]

  [[15 19 88 77]
   [90  5 67 42]]]


 [[[48 38  5  7]
   [91 94  3 41]]

  [[91 94  3 41]
   [91 94  3 41]]

  [[60 69 93  8]
   [91 94  3 41]]]


 [[[31 17 26 78]
   [39 76 33 84]]

  [[39 76 33 84]
   [39 76 33 84]]

  [[41 20 25 68]
   [39 76 33 84]]]], shape=(3, 3, 2, 4), dtype=int64)
[[72 56 67 16 90  5 67 42]
 [91 94  3 41 91 94  3 41]
 [41 20 25 68 39 76 33 84]]


InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0,3] = 3 is not in [0, 3) [Op:GatherV2] name: 

In [38]:
src = np.array([[[1,2,3],[4,5,6]],
                [[7,8,9],[10,11,12]]])

idx = np.array([[0,4],
                [7,11]])

# Wanted result
res = np.take(src, idx)
print(res)

[[ 1  5]
 [ 8 12]]


In [17]:
def LinkPredictor2(nodes_n,features_n,ch_gnn1,ch_gnn2,drop_rate):

    #Inputs
    raw_adj = Input(shape=(nodes_n,nodes_n),name='adj_mtx')
    adjmtx = To_Sparse()(raw_adj)
    nodefeat= Input(shape=(nodes_n, features_n), name='feature_mtx')
    index_train = Input(shape=(2,),dtype = tf.int64, name='index_pair_nodes')
    
    #Layers
    conv1 = spektral.layers.GeneralConv(channels= ch_gnn1, name='gcn1')([nodefeat,adjmtx])
    conv2 = spektral.layers.GeneralConv(channels= ch_gnn2, name='gcn2')([conv1,adjmtx])
    extracted = node_pair_xt()([index_train,conv2])
    rshp = Reshape((int(np.floor(extracted.shape[2]*extracted.shape[3])),))(extracted)
    
    dense = Dense(64)(rshp)
    dense = Dropout(drop_rate)(dense)
    dense = Dense(12)(dense)
    dense = Dropout(drop_rate)(dense)
    output_layer = Dense(2, 'softmax')(dense)
    
    #Model
    model = Model(inputs= [raw_adj, nodefeat, index_train ], outputs = output_layer)
       
    return model
   

In [6]:
#print(Ad_input[1])

no_links =np.where(Ad_input[1] == 0)
links =np.where(Ad_input[1] == 1)

for item in range(len(no_links[0])):
    if(item == 0):
        index_no_link = [no_links[0][item], no_links[1][item]]
    else:
        #print(index[0][item], index[1][item])
        index_no_link = np.vstack([index_no_link,  [no_links[0][item], no_links[1][item]] ])


for item in range(len(links[0])):
    if(item == 0):
        index_link = [links[0][item], links[1][item]]
    else:
        #print(index[0][item], index[1][item])
        index_link = np.vstack([index_link,  [links[0][item], links[1][item]] ])


In [23]:
link_pred.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])

link_pred.fit([Ad_input,Feat_input,index_train],y, epochs = 10,batch_size = 1)

link_pred.predict([Ad_input,Feat_input,index_train])

Epoch 1/10
10/10 [==============================] - 1s 2ms/step - loss: 6.0598e-19 - accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 0s 2ms/step - loss: 2.1427e-36 - accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 3.3561e-38 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 8.0975e-25 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 7.7118e-24 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 1.8441e-37 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 1.1003e-31 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 2ms/step - loss: 5.5276e-26 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 3.9713e-2

InvalidArgumentError: Graph execution error:

Detected at node 'model/reshape/Reshape' defined at (most recent call last):
    File "/home/nacho/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/nacho/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/nacho/anaconda3/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/nacho/anaconda3/lib/python3.8/asyncio/base_events.py", li

In [ ]:
class DataGeneratorLinkPredictor(keras.utils.Sequence):
    
    def __init__(self, adj_mtx, edg_lst, zer_lst, nod_emb, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.adj_mtx = adj_mtx
        self.edg_lst = edj_lst
        self.nod_emb = nod_emb
        self.shuffle = shuffle
        self.edg_list = edg_lst
        self.zer_list = zer_lst
        self.zeros  = zer_lst.shape[0]
        self.ones  = edg_lst.shape[0]
        self.on_epoch_end()
        

    def __len__(self):
        if(self.ones >= self.zeros):
            items_per_batch = int(np.floor(self.zeros / self.batch_size))
        else:
            items_per_batch = int(np.floor(self.ones / self.batch_size))

        return items_per_batch
            

    def __getitem__(self, index):
        
        index0 = self.indexes_0[index*self.batch_size:(index+1)*self.batch_size,]
        index1 = self.indexes_1[index*self.batch_size:(index+1)*self.batch_size,]

        # Generate data
        x_adj, y = self.__data_generation(index1,index0)

        return x_adj, y

    def on_epoch_end(self):
        ione = np.arange(self.ones)
        izero = np.arange(self.zeros)
        
        
        if self.shuffle == True:
            np.random.shuffle(ione)
            np.random.shuffle(izero)
        
        self.indexes_0 = self.zer_list[izero,]
        self.indexes_1 = self.edg_list[ione,]

    def __data_generation(self, index1, index0):

        for i in range(index0.shape[0]):
            # Store sample
            X = self.adj_mtx
            X[index0[i,]]=1
            X = np.expand_dims(X, axis=0)
            Y = 0
            if(i == 0):
                x_out = X
                y_out=0
            else:
                x_out = np.concatenate([x_out,X], axis=0)
                y_out = np.concatenat([y_out,Y])
                
        for i in range(index1.shape[0]):
            # Store sample
            X = self.adj_mtx
            X[index1[i,]]=0
            X = np.expand_dims(X, axis=0)
            Y = 1
            x_out = np.concatenate([x_out,X], axis=0)
            y_out = np.concatenat([y_out,Y])

            #shuffle
            x_out = x_out[np.random.shuffle(np.arange(x_out.shape[0])),,]


        return x_out, self.nod_emb, keras.utils.to_categorical(y_out, num_classes=self.n_classes)